In [ ]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <br>
    420-A56<br>
    <br>
</center>

In [ ]:
#
# imports utilitaires
#

%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time

from bs4 import BeautifulSoup
from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD

from tqdm.notebook import tqdm

#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.dataset.MovieLens as mvl
import helpers.WebScraping as scrap
import helpers.Clustering as clstr

from helpers.jupyter import display_html

# Exploration

**ATTENTION** il faut exécuter Pretraitement.ipynb avant celui-ci

In [ ]:
movies_df = pd.read_csv("dataset/movies_pretraitement.csv")

print("Movies", movies_df.shape)
clstr.show_types(movies_df)
clstr.show_na(movies_df)

print("Head")
display(movies_df.head().round(2))

<font class="answer">

* 9724 individus, 7 variables (*title* est en fait un identifiant).
* aucune valeur manquante  
* *genres* est de type qualitatif, les autres variables sont quantitatives. Cependant, en regardant de plus près les descriptions de MovieLens, on se rend compte que tout est en fait qualitatif (rating est un système d'étoile avec une granularité de $\frac{1}{2}$).

In [ ]:
# si jamais certains algos sont slow
movies_samples_df = movies_df.sample(frac=0.4)

print(movies_df.shape)
print(movies_samples_df.shape)

## Distributions

In [ ]:
coi = movies_df.columns.to_list()
coi.remove("movieId")
coi.remove("title")
coi.remove("genres")

print("Stats générales")
for c in coi:
    display(movies_df[c].describe().to_frame().T)

In [ ]:
coi = movies_df.columns.to_list()
coi.remove("movieId")
coi.remove("title")
coi.remove("genres")

clstr.show_distributions(movies_df[coi], num_cols=3, figsize=(10, 5))

<font class="answer">
    
* Les films sont surtout récent: peak autour de l'an 2000
* Les films sont majoritairement votés par peu de personnes. Distribution très alongée avec un très for peak dans les faibles valeurs.
* rating moyen et median semble similaire; la majorité des rating est entre 3 et 4.
* rating_min et rating_max plus difficile à interpreter. Fait probablement plus de sens de mettre ce genre de stats par user?  

In [ ]:
genres = {}

def genre_hist(genres_array):
    print(type(genres_array))
    print(genres_array)

movies_df.genres.apply(genre_hist)

# display(movies_df.genres)


# sns.countplot(x="genres", data=movies_df)
# plt.xticks(rotation=90)
# plt.show()

## Valeurs aberrantes

Si on se positione dans un contexte de sugestion, on peu assumer des films récent et populaire donc rating_count, year et rating_mean sont des bons candidats pour filtrer rating_min et rating_max sont trop confus pour tirer des conclusions

In [ ]:
g = sns.pairplot(data=movies_df, vars=["year", "rating_count", "rating_mean"], corner=True)
# g.fig.set_size_inches(9, 6)

In [ ]:
coi = movies_df.columns.to_list()
coi.remove("movieId")
coi.remove("title")
coi.remove("genres")
coi.remove("rating_median")
coi.remove("rating_min")
coi.remove("rating_max")

clstr.show_correlation(movies_df[coi], figsize=(4, 2.5))